<a href="https://colab.research.google.com/github/RekhaChandrasekaran/MLPractice/blob/master/MIMIC_4_EDA_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA on MIMIC-IV - Patient admissions and Diagnoses
Finding Readmissions and presence of newly diagnosed diseases and conditions

In [1]:
# parent data folder
parent_dir = '/content/drive/My Drive/MEANS_GIC_Hackathon/MIMIC_4_Data'
mimic_path =  '/content/drive/My Drive/MEANS_GIC_Hackathon/MIMIC_4_Data'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# imports

import sys
sys.path.insert(1, parent_dir)

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import pickle
import collections

import re
import os
import random
import warnings
warnings.filterwarnings("ignore")

# How many unique Patients?

In [4]:
patients_df = pd.read_csv(parent_dir + '/patients.csv')

In [5]:
patients_df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10002723,F,0,2128,2017 - 2019,NaN
1,10003939,M,0,2184,2008 - 2010,NaN
2,10004222,M,0,2161,2014 - 2016,NaN
3,10005325,F,0,2154,2011 - 2013,NaN
4,10007338,F,0,2153,2017 - 2019,NaN


[ ] Understand anchor_year and how that relates to admissions

In [6]:
patients_df.subject_id.nunique()

383220

# Number of Unique patients - 383220

# How many unique admissions?

In [8]:
admissions_df = pd.read_csv(parent_dir + '/admissions.csv')

In [9]:
admissions_df.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,NaN,DIRECT EMER.,PHYSICIAN REFERRAL,HOME,Other,?,NaN,WHITE,NaN,NaN,0
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,WHITE,NaN,NaN,0
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,WHITE,2131-01-26 22:19:00,2131-01-27 05:39:00,0


In [10]:
admissions_df.subject_id.nunique()

257366

In [11]:
admissions_df.hadm_id.nunique()

524520

# (Revised) Number of Patients - 257,366 with 524,520 admissions

# How many patients were re-admitted?

In [18]:
num_admissions = admissions_df.groupby(['subject_id'])['hadm_id'].count()

In [22]:
num_admissions.sort_values(ascending=False)

subject_id
15496609    238
15464144    185
10714009    163
16662316    142
15229574    130
           ... 
16024188      1
16024151      1
16024117      1
16024088      1
10000019      1
Name: hadm_id, Length: 257366, dtype: int64

**How did a patient get admitted 238 times? Are there duplicate records?**

In [24]:
admissions_df[admissions_df.subject_id==15496609]

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
204444,15496609,22456035,2163-06-26 23:47:00,2163-06-27 09:35:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2163-06-26 16:25:00,2163-06-27 09:35:00,0
204449,15496609,25283436,2163-07-12 18:01:00,2163-07-14 11:54:00,NaN,EU OBSERVATION,PHYSICIAN REFERRAL,NaN,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2163-07-12 15:22:00,2163-07-14 11:54:00,0
204471,15496609,29014836,2163-08-28 14:26:00,2163-08-28 20:11:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2163-08-28 10:51:00,2163-08-28 20:11:00,0
204498,15496609,23901857,2159-07-07 19:22:00,2159-07-08 09:50:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2159-07-07 16:48:00,2159-07-08 09:50:00,0
204554,15496609,25239597,2157-04-10 21:18:00,2157-04-11 05:05:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2157-04-10 19:08:00,2157-04-11 05:05:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234352,15496609,27901407,2163-06-08 23:32:00,2163-06-11 16:40:00,NaN,OBSERVATION ADMIT,EMERGENCY ROOM,HOME,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2163-06-08 20:09:00,2163-06-09 20:29:00,0
240938,15496609,28965033,2156-09-18 18:40:00,2156-09-20 13:00:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2156-09-18 16:11:00,2156-09-18 20:22:00,0
244212,15496609,23059133,2155-08-08 20:35:00,2155-08-10 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2155-08-08 16:23:00,2155-08-08 22:31:00,0
252567,15496609,26857156,2156-11-09 23:21:00,2156-11-15 12:39:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2156-11-09 19:09:00,2156-11-10 00:27:00,0


# Number of admissions

In [26]:
num_admissions.min(), num_admissions.max(), num_admissions.mean()

(1, 238, 2.0380314416045633)

**To keep things simple, lets take patients who got admitted 2-5 times**

In [30]:
filtered_subjects = num_admissions[np.logical_and(num_admissions>=2, num_admissions<=5)]

In [31]:
filtered_subjects

subject_id
10000032    4
10000084    2
10000117    2
10000730    2
10000826    3
           ..
19999442    2
19999464    2
19999625    2
19999828    2
19999840    2
Name: hadm_id, Length: 71175, dtype: int64

In [35]:
filtered_admissions = admissions_df[admissions_df['subject_id'].isin(filtered_subjects.index)]

In [37]:
filtered_admissions.shape

(194527, 15)

**Diagnosed Codes for each admission**

In [38]:
diagnoses_df = pd.read_csv(parent_dir + '/diagnoses_icd.csv')

In [39]:
diagnoses_df.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,11603789,25552978,3,65801,9
1,11603789,25552978,4,64881,9
2,11603789,25552978,5,V270,9
3,11603789,25552978,1,65221,9
4,11603789,25552978,2,64421,9


In [40]:
filtered_diagnoses = diagnoses_df[diagnoses_df['hadm_id'].isin(filtered_admissions['hadm_id'])]

In [41]:
filtered_diagnoses.shape

(1783646, 5)

In [49]:
grouped_admissions = filtered_diagnoses.groupby(['subject_id','hadm_id'])['icd_code'].unique()

In [50]:
grouped_admissions.shape

(164410,)

In [52]:
grouped_admissions.head(10)

subject_id  hadm_id 
10000032    22595853    [V1582  , 29680  , 5723   , 5715   , 78959  , ...
            22841357    [2875   , 78959  , 496    , 5715   , 07071  , ...
            25742920    [07054  , V462   , 2761   , V08    , 2767   , ...
            29079034    [2767   , 2761   , 3051   , V08    , 7994   , ...
10000084    23052089    [R441   , Z8546  , G3183  , F0280  , R296   , ...
            29888819    [R609   , F0280  , E785   , Z8546  , R4182  , ...
10000117    22927623    [Z87891 , F419   , R1310  , K449   , K31819 , ...
            27988844    [Y93K1  , E7800  , Z87442 , G43909 , S72012A, ...
10000730    24554295                                            [V3000  ]
            24810836                                            [7746   ]
Name: icd_code, dtype: object